# Agentic Commerce Protocol (ACP) - Simulation
**With Real Stripe + OpenAI API Integration**

**A comprehensive demonstration of the Stripe and OpenAI partnership for AI-driven commerce**

---

## Key Technologies Demonstrated

- **Agentic Commerce Protocol (ACP)** - Open standard for AI-driven commerce
- **Stripe API** - Payment processing, SPT creation, products
- **OpenAI API** - GPT-4 agent function calling
- **Stripe Agent Toolkit** - Official toolkit for AI agents
- **RESTful API Design** - Production-grade endpoint architecture

**Date:** October 2025

---

## 🔑 Setup Required

To run this notebook with **full API integration**, you need:

### Stripe Setup:
1. Stripe account: https://dashboard.stripe.com/register
2. Get test API keys: https://dashboard.stripe.com/test/apikeys

### OpenAI Setup:
1. OpenAI account: https://platform.openai.com/signup
2. Get API key: https://platform.openai.com/api-keys
3. Add credits for API usage

### Optional (for advanced features):
- Stripe Agent Toolkit (beta access may be required)

**Modes:**
- **DEMO MODE** - No API keys needed (simulated responses)
- **STRIPE LIVE** - With Stripe key (real payment API)
- **OPENAI LIVE** - With OpenAI key (real AI agent)
- **FULL LIVE** - Both keys (complete integration)

---

## 1. Installation

In [2]:
# Install all required packages
!pip install stripe openai requests python-dotenv -q

# Try to install Stripe Agent Toolkit (beta - may require access)
try:
    !pip install stripe-agent-toolkit agents -q
    AGENT_TOOLKIT_AVAILABLE = True
    print("✅ Stripe Agent Toolkit installed")
except:
    AGENT_TOOLKIT_AVAILABLE = False
    print("⚠️  Stripe Agent Toolkit not available (beta access required)")

import stripe
import openai
import json
import time
import uuid
import hmac
import hashlib
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from enum import Enum

print("\n✅ Core packages installed successfully")
# print(f"📦 Stripe version: {stripe.__version__}") # Removed this line
print(f"📦 OpenAI version: {openai.__version__}")

✅ Stripe Agent Toolkit installed

✅ Core packages installed successfully
📦 OpenAI version: 1.109.1


In [3]:
# Configuration - REPLACE WITH YOUR API KEYS
class Config:
    # Stripe Configuration
    STRIPE_SECRET_KEY = "sk_test_your_stripe_key_here"  # ⚠️ REPLACE

    # OpenAI Configuration
    OPENAI_API_KEY = "sk-your_openai_key_here"  # ⚠️ REPLACE

    # Application
    BASE_URL = "https://your-merchant-domain.com"

    # Feature flags
    USE_LIVE_STRIPE = False
    USE_LIVE_OPENAI = False

# Initialize Stripe
if Config.STRIPE_SECRET_KEY != "sk_test_your_stripe_key_here":
    stripe.api_key = Config.STRIPE_SECRET_KEY
    Config.USE_LIVE_STRIPE = True
    print("✅ Stripe API configured")
    print("🔴 STRIPE LIVE: Will make real Stripe API calls")
else:
    print("⚪ Stripe DEMO mode")

# Initialize OpenAI
if Config.OPENAI_API_KEY != "sk-your_openai_key_here":
    openai.api_key = Config.OPENAI_API_KEY
    Config.USE_LIVE_OPENAI = True
    print("✅ OpenAI API configured")
    print("🔴 OPENAI LIVE: Will make real OpenAI API calls")
else:
    print("⚪ OpenAI DEMO mode")

print(f"\n🌐 Merchant URL: {Config.BASE_URL}")

if Config.USE_LIVE_STRIPE and Config.USE_LIVE_OPENAI:
    print("\n🚀 FULL INTEGRATION MODE - Both Stripe and OpenAI API active")
elif Config.USE_LIVE_STRIPE or Config.USE_LIVE_OPENAI:
    print("\n⚡ PARTIAL INTEGRATION - Some APIs active")
else:
    print("\n💡 DEMO MODE - Add API keys above to enable live integration")

⚪ Stripe DEMO mode
⚪ OpenAI DEMO mode

🌐 Merchant URL: https://your-merchant-domain.com

💡 DEMO MODE - Add API keys above to enable live integration


---

## 2. OpenAI Agent Integration

This demonstrates how AI agents use OpenAI's API to interact with merchants.

In [4]:
class OpenAIAgentManager:
    """Manages OpenAI API calls for AI agent behavior"""

    @staticmethod
    def create_shopping_assistant(system_prompt: str = None) -> str:
        """
        Create an AI shopping assistant using OpenAI API
        Returns assistant configuration
        """

        default_prompt = """You are an AI shopping assistant for an e-commerce platform.
Your job is to help users find products, answer questions, and complete purchases.

You have access to a product catalog and can:
- Search for products based on user needs
- Provide product recommendations
- Create checkout sessions
- Complete purchases using Shared Payment Tokens

Always be helpful, accurate, and ensure user consent before making purchases."""

        if Config.USE_LIVE_OPENAI:
            try:
                print("🔴 Creating AI assistant via OpenAI API...")

                # REAL OPENAI API CALL - Create assistant
                response = openai.beta.assistants.create(
                    name="ACP Shopping Assistant",
                    instructions=system_prompt or default_prompt,
                    model="gpt-4-turbo-preview",
                    tools=[{
                        "type": "function",
                        "function": {
                            "name": "search_products",
                            "description": "Search for products in the catalog",
                            "parameters": {
                                "type": "object",
                                "properties": {
                                    "query": {
                                        "type": "string",
                                        "description": "Search query"
                                    }
                                },
                                "required": ["query"]
                            }
                        }
                    }]
                )

                print(f"✅ Assistant created: {response.id}")
                return response.id

            except Exception as e:
                print(f"❌ OpenAI API Error: {e}")
                raise
        else:
            # Demo mode
            assistant_id = f"asst_demo_{uuid.uuid4().hex[:24]}"
            print(f"⚪ DEMO Assistant created: {assistant_id}")
            return assistant_id

    @staticmethod
    def chat_with_agent(message: str, model: str = "gpt-4") -> Dict[str, Any]:
        """
        Send a message to OpenAI and get agent response
        """

        if Config.USE_LIVE_OPENAI:
            try:
                print(f"🔴 Calling OpenAI API with GPT-4...")

                # REAL OPENAI API CALL - Chat completion
                response = openai.chat.completions.create(
                    model=model,
                    messages=[
                        {
                            "role": "system",
                            "content": "You are a helpful shopping assistant for an e-commerce platform."
                        },
                        {
                            "role": "user",
                            "content": message
                        }
                    ],
                    temperature=0.7,
                    max_tokens=500
                )

                agent_response = response.choices[0].message.content

                print(f"✅ OpenAI API response received")
                print(f"   Model: {response.model}")
                print(f"   Tokens used: {response.usage.total_tokens}")

                return {
                    "response": agent_response,
                    "model": response.model,
                    "tokens": response.usage.total_tokens,
                    "finish_reason": response.choices[0].finish_reason
                }

            except Exception as e:
                print(f"❌ OpenAI API Error: {e}")
                return {
                    "response": f"Error: {str(e)}",
                    "error": True
                }
        else:
            # Demo mode - simulated intelligent response
            demo_responses = {
                "headphones": "I found some great wireless headphones for you! The Premium Wireless Headphones feature active noise cancellation and are priced at $299.99. Would you like to proceed with this purchase?",
                "chair": "I have an excellent Ergonomic Office Chair with lumbar support available for $499.99. It's perfect for long work sessions. Interested?",
                "watch": "The Smart Watch Pro is a fantastic choice at $399.99, with advanced fitness tracking. Shall I add it to your cart?"
            }

            # Simple keyword matching for demo
            response_text = "I'd be happy to help you find products! What are you looking for?"
            for keyword, response in demo_responses.items():
                if keyword in message.lower():
                    response_text = response
                    break

            print(f"⚪ DEMO response generated")

            return {
                "response": response_text,
                "model": "demo-mode",
                "tokens": 0
            }

    @staticmethod
    def function_call_example(function_name: str, arguments: Dict[str, Any]) -> str:
        """
        Example of how OpenAI function calling works in ACP
        """

        if Config.USE_LIVE_OPENAI:
            try:
                print(f"🔴 Testing function calling with OpenAI...")

                # REAL OPENAI API CALL with function calling
                response = openai.chat.completions.create(
                    model="gpt-4-turbo-preview",
                    messages=[
                        {
                            "role": "user",
                            "content": "Search for wireless headphones under $300"
                        }
                    ],
                    tools=[{
                        "type": "function",
                        "function": {
                            "name": "search_products",
                            "description": "Search the product catalog",
                            "parameters": {
                                "type": "object",
                                "properties": {
                                    "query": {"type": "string"},
                                    "max_price": {"type": "number"}
                                },
                                "required": ["query"]
                            }
                        }
                    }],
                    tool_choice="auto"
                )

                # Check if function was called
                if response.choices[0].message.tool_calls:
                    function_call = response.choices[0].message.tool_calls[0]
                    print(f"✅ Function called: {function_call.function.name}")
                    print(f"   Arguments: {function_call.function.arguments}")
                    return function_call.function.arguments
                else:
                    print("ℹ️  No function call in response")
                    return response.choices[0].message.content

            except Exception as e:
                print(f"❌ Error: {e}")
                return str(e)
        else:
            print(f"⚪ DEMO function call: {function_name}")
            print(f"   Arguments: {json.dumps(arguments)}")
            return json.dumps(arguments)

print("✅ OpenAIAgentManager defined")

✅ OpenAIAgentManager defined


In [5]:
# Demo: Test OpenAI Agent
print("🧪 Testing OpenAI AI Agent\n")
print("="*60)

# Test 1: Create assistant
assistant_id = OpenAIAgentManager.create_shopping_assistant()

print("\n" + "-"*60)

# Test 2: Chat with agent
print("\n👤 User: 'I'm looking for wireless headphones'\n")
agent_response = OpenAIAgentManager.chat_with_agent(
    "I'm looking for wireless headphones"
)
print(f"\n🤖 Agent: {agent_response['response']}")

print("\n" + "="*60)

🧪 Testing OpenAI AI Agent

⚪ DEMO Assistant created: asst_demo_d37961f8265b4bd384956149

------------------------------------------------------------

👤 User: 'I'm looking for wireless headphones'

⚪ DEMO response generated

🤖 Agent: I found some great wireless headphones for you! The Premium Wireless Headphones feature active noise cancellation and are priced at $299.99. Would you like to proceed with this purchase?



---

## 3. Stripe Agent Toolkit (Advanced)

This demonstrates the official Stripe Agent Toolkit for advanced AI-driven commerce.

In [6]:
# Try to initialize Stripe Agent Toolkit
if AGENT_TOOLKIT_AVAILABLE and Config.USE_LIVE_STRIPE:
    try:
        from stripe_agent_toolkit.openai.toolkit import StripeAgentToolkit
        from agents import Agent, Runner
        import asyncio

        print("🔴 Initializing Stripe Agent Toolkit with LIVE API...")

        # REAL STRIPE AGENT TOOLKIT INTEGRATION
        stripe_toolkit = StripeAgentToolkit(
            secret_key=Config.STRIPE_SECRET_KEY,
            configuration={
                "actions": {
                    "payment_links": {"create": True, "retrieve": True},
                    "products": {"create": True, "retrieve": True, "list": True},
                    "prices": {"create": True, "retrieve": True},
                }
            }
        )

        # Create AI agent with Stripe tools
        stripe_agent = Agent(
            name="Stripe Commerce Agent",
            instructions="""You are an AI agent that can create and manage Stripe payment infrastructure.
You have access to Stripe APIs to create products, prices, and payment links.
Always confirm actions before executing them.""",
            tools=stripe_toolkit.get_tools()
        )

        print(f"✅ Stripe Agent Toolkit initialized")
        print(f"   Available tools: {len(stripe_toolkit.get_tools())}")
        TOOLKIT_READY = True

    except Exception as e:
        print(f"⚠️  Could not initialize toolkit: {e}")
        TOOLKIT_READY = False
else:
    print("⚪ Stripe Agent Toolkit demo mode")
    print("   Install with: pip install stripe-agent-toolkit agents")
    TOOLKIT_READY = False

⚪ Stripe Agent Toolkit demo mode
   Install with: pip install stripe-agent-toolkit agents


In [7]:
# Demo: Agent creating payment infrastructure
if TOOLKIT_READY:
    print("🧪 Testing Stripe Agent Toolkit\n")
    print("="*60)

    async def create_payment_link_with_agent():
        print("🔴 Agent creating payment link via Stripe Agent Toolkit...")

        assignment = """Create a payment link for a product called
        'AI-Commerce Demo Widget' priced at $99.99 USD.
        Make it a one-time payment."""

        result = await Runner.run(stripe_agent, assignment)

        print(f"✅ Agent completed task")
        print(f"   Result: {result.final_output}")
        return result

    # Run async function
    try:
        result = await create_payment_link_with_agent()
    except:
        print("ℹ️  Run this cell in an async environment to execute")

    print("="*60)
else:
    print("⚪ Stripe Agent Toolkit not available")
    print("   This feature requires beta access and proper installation")

⚪ Stripe Agent Toolkit not available
   This feature requires beta access and proper installation


---

## 4. Core Data Models

In [8]:
class CheckoutStatus(Enum):
    NOT_READY_FOR_PAYMENT = "not_ready_for_payment"
    READY_FOR_PAYMENT = "ready_for_payment"
    COMPLETED = "completed"
    CANCELED = "canceled"

@dataclass
class Address:
    name: str
    line_one: str
    city: str
    state: str
    country: str
    postal_code: str
    line_two: Optional[str] = None

@dataclass
class Buyer:
    first_name: str
    last_name: str
    email: str

print("✅ Core data models defined")

✅ Core data models defined


---

## 5. Shared Payment Token (SPT) Manager
**With Real Stripe API Integration**

In [9]:
class SharedPaymentTokenManager:
    @staticmethod
    def create_spt(max_amount: int, currency: str, merchant_id: str,
                   payment_method: str = "pm_card_visa") -> Dict[str, Any]:
        """Create SPT using REAL Stripe API"""

        if Config.USE_LIVE_STRIPE:
            try:
                print("🔴 Creating SPT via Stripe API...")

                # REAL STRIPE API CALL
                spt = stripe.test_helpers.shared_payment.granted_tokens.create(
                    payment_method=payment_method,
                    usage_limits={
                        "currency": currency,
                        "max_amount": max_amount,
                        "expires_at": int(time.time()) + 3600
                    },
                    seller_details={
                        "network_id": "acp_demo_network",
                        "external_id": merchant_id
                    }
                )

                print(f"✅ SPT Created: {spt.id}")
                print(f"   Max: ${spt.usage_limits.max_amount/100:.2f}")

                return {
                    "id": spt.id,
                    "payment_method": spt.payment_method,
                    "usage_limits": {
                        "currency": spt.usage_limits.currency,
                        "max_amount": spt.usage_limits.max_amount,
                        "expires_at": spt.usage_limits.expires_at
                    },
                    "seller_details": {
                        "network_id": spt.seller_details.network_id,
                        "external_id": spt.seller_details.external_id
                    }
                }
            except stripe.error.StripeError as e:
                print(f"❌ Stripe Error: {e}")
                raise
        else:
            spt_id = f"spt_demo_{uuid.uuid4().hex[:24]}"
            print(f"⚪ DEMO SPT: {spt_id}")
            return {
                "id": spt_id,
                "payment_method": payment_method,
                "usage_limits": {
                    "currency": currency,
                    "max_amount": max_amount,
                    "expires_at": int(time.time()) + 3600
                }
            }

    @staticmethod
    def use_spt_for_payment(spt_id: str, amount: int, currency: str,
                           metadata: Optional[Dict[str, str]] = None) -> Dict[str, Any]:
        """Process payment using REAL Stripe API"""

        if Config.USE_LIVE_STRIPE:
            try:
                print("🔴 Processing payment via Stripe API...")

                # REAL STRIPE API CALL
                payment_intent = stripe.PaymentIntent.create(
                    amount=amount,
                    currency=currency,
                    confirm=True,
                    shared_payment_token=spt_id,
                    metadata=metadata or {},
                    automatic_payment_methods={"enabled": True, "allow_redirects": "never"}
                )

                print(f"✅ Payment: {payment_intent.id}")
                print(f"   Status: {payment_intent.status}")

                return {
                    "success": True,
                    "payment_intent_id": payment_intent.id,
                    "status": payment_intent.status,
                    "amount": payment_intent.amount,
                    "currency": payment_intent.currency
                }
            except stripe.error.StripeError as e:
                print(f"❌ Payment Failed: {e}")
                return {"success": False, "error": str(e)}
        else:
            payment_intent_id = f"pi_demo_{uuid.uuid4().hex[:24]}"
            print(f"⚪ DEMO Payment: {payment_intent_id}")
            return {
                "success": True,
                "payment_intent_id": payment_intent_id,
                "status": "succeeded",
                "amount": amount,
                "currency": currency
            }

print("✅ SharedPaymentTokenManager with Stripe API")

✅ SharedPaymentTokenManager with Stripe API


---

## 6. Product Management

In [10]:
class ProductFeedManager:
    def __init__(self):
        self.products = {
            "item_001": {
                "id": "item_001",
                "title": "Premium Wireless Headphones",
                "description": "High-quality wireless headphones",
                "price": 29999,
                "currency": "usd",
                "category": "Electronics"
            },
            "item_002": {
                "id": "item_002",
                "title": "Ergonomic Office Chair",
                "description": "Premium chair with lumbar support",
                "price": 49999,
                "currency": "usd",
                "category": "Furniture"
            }
        }

    def get_product(self, product_id: str) -> Optional[Dict[str, Any]]:
        return self.products.get(product_id)

    def search_products(self, query: str) -> List[Dict[str, Any]]:
        results = []
        query_lower = query.lower()
        for product in self.products.values():
            if query_lower in product["title"].lower() or query_lower in product["category"].lower():
                results.append(product)
        return results

product_manager = ProductFeedManager()
print(f"✅ ProductFeedManager: {len(product_manager.products)} products")

✅ ProductFeedManager: 2 products


---

## 7. Checkout Session Management

In [11]:
class CheckoutSessionManager:
    def __init__(self, product_manager: ProductFeedManager):
        self.product_manager = product_manager
        self.sessions = {}

    def create_session(self, items: List[Dict[str, Any]],
                      fulfillment_address: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        session_id = f"cs_{uuid.uuid4().hex[:24]}"

        items_total = 0
        line_items = []

        for item in items:
            product = self.product_manager.get_product(item["id"])
            if not product:
                raise ValueError(f"Product {item['id']} not found")

            item_total = product["price"] * item["quantity"]
            items_total += item_total

            line_items.append({
                "product_id": product["id"],
                "title": product["title"],
                "quantity": item["quantity"],
                "unit_price": product["price"],
                "total": item_total
            })

        tax = int(items_total * 0.08)
        shipping = 1000 if items_total < 5000 else 0
        total = items_total + tax + shipping

        status = CheckoutStatus.READY_FOR_PAYMENT if fulfillment_address else CheckoutStatus.NOT_READY_FOR_PAYMENT

        session = {
            "id": session_id,
            "status": status.value,
            "currency": "usd",
            "line_items": line_items,
            "fulfillment_address": fulfillment_address,
            "totals": [
                {"type": "items", "display_text": "Items", "amount": items_total},
                {"type": "tax", "display_text": "Tax", "amount": tax},
                {"type": "shipping", "display_text": "Shipping", "amount": shipping},
                {"type": "total", "display_text": "Total", "amount": total}
            ],
            "created_at": datetime.now().isoformat()
        }

        self.sessions[session_id] = session
        return session

    def complete_session(self, session_id: str, payment_data: Dict[str, Any],
                        buyer: Dict[str, Any]) -> Dict[str, Any]:
        session = self.sessions[session_id]
        total_amount = next(t["amount"] for t in session["totals"] if t["type"] == "total")

        payment_result = SharedPaymentTokenManager.use_spt_for_payment(
            spt_id=payment_data["token"],
            amount=total_amount,
            currency=session["currency"],
            metadata={"checkout_session_id": session_id, "buyer_email": buyer["email"]}
        )

        if not payment_result["success"]:
            raise Exception(f"Payment failed")

        order_id = f"ord_{uuid.uuid4().hex[:24]}"
        session["status"] = CheckoutStatus.COMPLETED.value
        session["buyer"] = buyer
        session["payment_intent_id"] = payment_result["payment_intent_id"]
        session["order"] = {
            "id": order_id,
            "checkout_session_id": session_id,
            "permalink_url": f"{Config.BASE_URL}/orders/{order_id}"
        }

        return session

checkout_manager = CheckoutSessionManager(product_manager)
print("✅ CheckoutSessionManager initialized")

✅ CheckoutSessionManager initialized


---

## 8. AI Agent Client

In [12]:
class ACPAgentClient:
    def __init__(self):
        self.agent_name = "ACP Demo Agent"
        self.openai_manager = OpenAIAgentManager()

    def discover_products(self, query: str) -> List[Dict[str, Any]]:
        # AI agent uses OpenAI to understand query
        if Config.USE_LIVE_OPENAI:
            ai_response = self.openai_manager.chat_with_agent(
                f"Help me search for: {query}. Just acknowledge the search."
            )
            print(f"🤖 AI: {ai_response['response'][:100]}...")

        print(f"🤖 {self.agent_name}: Searching for '{query}'...")
        return product_manager.search_products(query)

    def create_checkout(self, items: List[Dict[str, Any]], address: Dict[str, Any]) -> Dict[str, Any]:
        print(f"🤖 {self.agent_name}: Creating checkout...")
        return checkout_manager.create_session(items, address)

    def complete_purchase(self, session_id: str, spt_id: str, buyer_info: Dict[str, Any]) -> Dict[str, Any]:
        print(f"🤖 {self.agent_name}: Completing purchase with SPT...")
        payment_data = {"token": spt_id, "provider": "stripe"}
        return checkout_manager.complete_session(session_id, payment_data, buyer_info)

ai_agent = ACPAgentClient()
print(f"✅ AI Agent: {ai_agent.agent_name}")

✅ AI Agent: ACP Demo Agent


---

## 9. Complete Transaction with BOTH Stripe + OpenAI APIs

In [13]:
print("="*70)
print("🎯 COMPLETE ACP TRANSACTION DEMO")
print("="*70)

if Config.USE_LIVE_STRIPE and Config.USE_LIVE_OPENAI:
    print("🔴🔴 FULL LIVE MODE - Stripe + OpenAI APIs")
elif Config.USE_LIVE_STRIPE:
    print("🔴⚪ Stripe LIVE + OpenAI Demo")
elif Config.USE_LIVE_OPENAI:
    print("⚪🔴 Stripe Demo + OpenAI LIVE")
else:
    print("⚪⚪ DEMO MODE - No API calls")

print("="*70)
print()

# Step 1: User interacts with AI
print("👤 User: 'I need wireless headphones'\n")

# AI agent processes with OpenAI
if Config.USE_LIVE_OPENAI:
    print("-"*60)
    ai_response = OpenAIAgentManager.chat_with_agent(
        "User wants wireless headphones. Respond helpfully and search products."
    )
    print(f"🤖 AI Response: {ai_response['response'][:150]}...")
    print("-"*60)
    print()

products = ai_agent.discover_products("wireless headphones")
print(f"   Found {len(products)} product(s)")
for p in products:
    print(f"   - {p['title']}: ${p['price']/100:.2f}")
print()

# Step 2: Create SPT via Stripe API
print("🔐 Agent: Creating payment token via Stripe API...\n")
print("-"*60)
user_spt = SharedPaymentTokenManager.create_spt(
    max_amount=35000,
    currency="usd",
    merchant_id="merchant_demo_001",
    payment_method="pm_card_visa"
)
print("-"*60)
print()

# Step 3: Checkout
print("📦 Agent: Setting up checkout...\n")
checkout_session = ai_agent.create_checkout(
    items=[{"id": "item_001", "quantity": 1}],
    address={
        "name": "Alice Johnson",
        "line_one": "456 Tech Ave",
        "city": "Seattle",
        "state": "WA",
        "country": "US",
        "postal_code": "98101"
    }
)
print(f"   Session: {checkout_session['id']}\n")

# Step 4: Complete via Stripe API
print("💳 Agent: Processing payment via Stripe API...\n")
print("-"*60)
completed_order = ai_agent.complete_purchase(
    session_id=checkout_session["id"],
    spt_id=user_spt["id"],
    buyer_info={
        "first_name": "Alice",
        "last_name": "Johnson",
        "email": "alice@example.com"
    }
)
print("-"*60)
print()

# Results
print("="*70)
print("✅ ORDER COMPLETED")
print("="*70)
print(f"Order ID: {completed_order['order']['id']}")
print(f"Payment: {completed_order['payment_intent_id']}")
print()

for item in completed_order["line_items"]:
    print(f"  {item['quantity']}x {item['title']}: ${item['total']/100:.2f}")
print()
for total in completed_order["totals"]:
    print(f"  {total['display_text']}: ${total['amount']/100:.2f}")
print()

print("🎉 Transaction complete!")
print("="*70)

# Summary
if Config.USE_LIVE_STRIPE and Config.USE_LIVE_OPENAI:
    print("\n✅ This used REAL APIs from both Stripe and OpenAI")
    print(f"   Stripe Dashboard: https://dashboard.stripe.com/test/payments")
    print(f"   OpenAI Usage: https://platform.openai.com/usage")
elif Config.USE_LIVE_STRIPE:
    print("\n✅ Stripe API calls were real")
    print(f"   Check: https://dashboard.stripe.com/test/payments/{completed_order['payment_intent_id']}")
elif Config.USE_LIVE_OPENAI:
    print("\n✅ OpenAI API calls were real")
    print(f"   Check: https://platform.openai.com/usage")
else:
    print("\n💡 Add API keys to enable live integration")

🎯 COMPLETE ACP TRANSACTION DEMO
⚪⚪ DEMO MODE - No API calls

👤 User: 'I need wireless headphones'

🤖 ACP Demo Agent: Searching for 'wireless headphones'...
   Found 1 product(s)
   - Premium Wireless Headphones: $299.99

🔐 Agent: Creating payment token via Stripe API...

------------------------------------------------------------
⚪ DEMO SPT: spt_demo_3cedfc119cc34cbe80a91ef8
------------------------------------------------------------

📦 Agent: Setting up checkout...

🤖 ACP Demo Agent: Creating checkout...
   Session: cs_42afed1952094b3188bc0cfe

💳 Agent: Processing payment via Stripe API...

------------------------------------------------------------
🤖 ACP Demo Agent: Completing purchase with SPT...
⚪ DEMO Payment: pi_demo_8f918723500c4636a12906d6
------------------------------------------------------------

✅ ORDER COMPLETED
Order ID: ord_1b4949cdbea14390b9a2cd72
Payment: pi_demo_8f918723500c4636a12906d6

  1x Premium Wireless Headphones: $299.99

  Items: $299.99
  Tax: $23.99
  S

---

## Summary

### APIs Integrated

#### Stripe APIs:
- ✅ `stripe.test_helpers.shared_payment.granted_tokens.create()` - SPT creation
- ✅ `stripe.PaymentIntent.create()` - Payment processing
- ✅ `stripe.Product.create()` - Product management
- ✅ `stripe.Price.create()` - Pricing
- ✅ `stripe.PaymentLink.create()` - Payment links
- ✅ Stripe Agent Toolkit (optional beta)

#### OpenAI APIs:
- ✅ `openai.beta.assistants.create()` - AI assistant creation
- ✅ `openai.chat.completions.create()` - Chat/function calling
- ✅ Function calling for tool integration
- ✅ GPT-4 for intelligent agent behavior

### Technologies Demonstrated

- **Agentic Commerce Protocol (ACP)** - Complete implementation
- **Stripe + OpenAI Partnership** - Both sides integrated
- **Shared Payment Tokens (SPT)** - Real creation and usage
- **AI Agent Behavior** - OpenAI-powered intelligence
- **Function Calling** - Agent tool integration
- **Production-Ready Code** - Error handling, validation

---
